In [17]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.SIRModel as sir

In [2]:
def read_graph(filename):
    
    g = nx.Graph()
    
    f = open(filename)
    for l in f:
        l = l.rstrip().replace(" ", ";").replace(",", ";").replace("\t", ";").split(";")
        g.add_edge(l[0], l[1])
    return g

g = read_graph("Dataset/socfb_Sample.csv")

In [18]:
G_random = nx.gnp_random_graph(6621,0.0111)

In [19]:
# Model Selection
model = sir.SIRModel(G_random)

# Model Configuration
config = mc.Configuration()
config.add_model_parameter('beta', 0.001)
config.add_model_parameter('gamma', 0.01)
config.add_model_parameter("percentage_infected", 0.05)
model.set_initial_status(config)

# Simulation
iterations = model.iteration_bunch(500)

In [ ]:
from bokeh.io import output_notebook, show
from ndlib.viz.bokeh.DiffusionTrend import DiffusionTrend

output_notebook()

viz = DiffusionTrend(model, iterations)
p = viz.plot(width=400, height=400)
show(p)

In [21]:
from ndlib.viz.bokeh.DiffusionPrevalence import DiffusionPrevalence

viz2 = DiffusionPrevalence(model, iterations)
p2 = viz2.plot(width=400, height=400)

show(p2)

In [22]:
from ndlib.viz.bokeh.MultiPlot import MultiPlot

vm = MultiPlot()
vm.add_plot(p)
vm.add_plot(p2)
m = vm.plot()
show(m)

In [23]:
import ndlib.models.epidemics.SISModel as sis
import ndlib.models.epidemics.SIModel as si
import ndlib.models.epidemics.ThresholdModel as th

vm = MultiPlot()
vm.add_plot(p)

# SIS
sis_model = sis.SISModel(G_random)
config = mc.Configuration()
config.add_model_parameter('beta', 0.001)
config.add_model_parameter('lambda', 0.01)
config.add_model_parameter("percentage_infected", 0.05)
sis_model.set_initial_status(config)
iterations = sis_model.iteration_bunch(500)
viz = DiffusionTrend(sis_model, iterations)
p3 = viz.plot(width=400, height=400)
vm.add_plot(p3)

# SI
si_model = si.SIModel(G_random)
config = mc.Configuration()
config.add_model_parameter('beta', 0.001)
config.add_model_parameter("percentage_infected", 0.05)
si_model.set_initial_status(config)
iterations = si_model.iteration_bunch(500)
viz = DiffusionTrend(si_model, iterations)
p4 = viz.plot(width=400, height=400)
vm.add_plot(p4)

# Threshold
th_model = th.ThresholdModel(G_random)
config = mc.Configuration()

# Set individual node threshold
threshold = 0.40
for n in G_random.nodes():
    config.add_node_configuration("threshold", n, threshold)

config.add_model_parameter("percentage_infected", 0.30)
th_model.set_initial_status(config)
iterations = th_model.iteration_bunch(300)
viz = DiffusionTrend(th_model, iterations)
p5 = viz.plot(width=400, height=400)
vm.add_plot(p5)

m = vm.plot()
show(m)